In [1]:
import numpy as np
import sys
from matplotlib import pyplot as plt
import SimpleITK as sitk

import torch



sys.path.append("../../")

from scripts.USImageCV.utils.bunny import bunny_mask_predict, load_bunny_model

 

%load_ext autoreload
%autoreload 2

# Load pre-trained segmentation model

In [2]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

trunk_model = load_bunny_model('./model/branching_point_model.pth',device)
# vessel_model = load_bunny_model('../../scripts/USImageCV/model_weights/vessel_model.pth',device)

In [44]:
for img_idx in range(10):
    print(img_idx)
    img = sitk.ReadImage('./data/img_{}.nii.gz'.format(img_idx))
    spacing, origin = img.GetSpacing(),img.GetSpacing()
    img = sitk.GetArrayFromImage(img)

    masks = []
    for i in range(len(img)):
        frame = img[i]
        masks.append(bunny_mask_predict(trunk_model,frame,device))
    masks = np.array(masks)


    pred = sitk.GetImageFromArray(masks)
    pred.SetSpacing(spacing)
    pred.SetOrigin(origin)
    sitk.WriteImage(pred,'./data/branching-point/Branching_prediction_{}.nrrd'.format(img_idx))
